# Lookup data from SQLite emonitor database

In [1]:
# packages
import os
import sqlite3
from emonitor.tools import db_path, history, live
# ploting
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.palettes import Category10
colors = Category10[10]
output_notebook()

Loading BokehJS ...

## SQLite database file

In [2]:
# file
name = "fake"
fil = db_path(name)
if os.path.exists(fil):
    print(fil)
else:
    raise Exception("sqlite file not found")

/home/adam/.emonitor/data/fake.db


Queries are submitted to the database using `emonitor.tools.history()` or `emonitor.tools.live()`.

## Live data

Load the database file and submit a query for all data taken in the past 12 hours.  Returns a `pandas.DataFrame()`. 

The index timestamp is in UTC; use the keyword argument 'tz' to convert.

Use Bokeh to create an interactive plot of the pressure data.

In [3]:
# time range
delta = {"hours":2}

# data
conn = sqlite3.connect(fil)
df = live(conn, delta, tz='Europe/Berlin')
conn.close()

# figure
fig = figure(plot_height=300, plot_width=500,
             x_axis_type="datetime")

# plotting with localized data
tvals = df.index.tz_localize(None)
for i, col in enumerate(df.columns):
    fig.line(tvals, df[col], line_color=colors[i], legend_label=col)

# format
fig.yaxis.axis_label = "pressure (mbar)"
fig.legend.click_policy="hide"
show(fig)

## History lookup

Submit a query for data in a given time range.

In [4]:
# datetime range
start = (2018, 1, 10)
end = (2020, 4, 28)

# data
conn = sqlite3.connect(fil)
df2 = history(conn, start, end)
conn.close()

# figure
fig = figure(plot_height=300, plot_width=500,
             x_axis_type="datetime", y_axis_type="log")

# plotting UTC time data
for i, col in enumerate(df2.columns):
    fig.line(df2.index, df2[col], line_color=colors[i], legend_label=col)

# format
fig.yaxis.axis_label = "pressure (mbar)"
fig.legend.click_policy="hide"
show(fig)